# 作业

**Q1: 怎么在ip被封之后实现自动更换代理池内的代理？**

A1: 用random.choice 随机选取ip

**Q2: 如何用一句通俗的语言解释清楚request、beautifulsoup和selenium三者与浏览器之间的关系？**

A2: 
- 浏览器获取网页第一步：request获取的html内容;
- 浏览器获取网页第二步：beautifulsoup解析静态html;
- 浏览器获取网页第三步：使用内核动态解析js、php，selenium调用driver模仿此步骤，所见即所得。

**Q3: 构建好代理池后，如何在一次爬虫中自动切换代理？ 比如代理无效，或者代理ip被封，这时自动切换下一个ip。**

A3: 首先你要有一个ip代理池（如果比较豪可以自己买真实ip自行搭建，好处独享有技术门槛,或者找第三方ip代理商对接,好吃廉价,但ip不独享）， 真实ip需要自己写程序来代理转发，第三方ip代理商则会提供相关转发API,直接调用就可以，这东西没什么技术难度 

**Q4: ip_list.append(f'{protpcol}://{ip}:{port}')这里的f是格式化？**

A4:

- f-string在形式上是以 f 或 F 修饰符引领的字符串（f'xxx' 或 F'xxx'），以大括号 {} 标明被替换的字段；

1. 从代理*ip*网站爬取*IP*地址及端口号并储存
2. 验证*ip*是否能用
3. 格式化*ip*地址
4. 在*requests*中使用代理*ip*爬取网站

# 小项目
1. 挑战项目：模拟登录丁香园，并抓取论坛页面所有的人员基本信息与回复帖子内容。
2. 丁香园论坛：[http://www.dxy.cn/bbs/thread/626626#626626](http://www.dxy.cn/bbs/thread/626626#626626) 。

In [2]:
import requests, json, re, random,time
from bs4 import BeautifulSoup
from selenium import webdriver
from lxml import etree
import time

In [3]:
"""
使用selenium进行模拟登陆
1.初始化ChromDriver
2.打开163登陆页面
3.找到用户名的输入框，输入用户名
4.找到密码框，输入密码
5.提交用户信息
"""
name = '15821480867'
passwd = 'a_123456'
driver = webdriver.Chrome()
driver.get('http://www.dxy.cn/bbs/thread/626626#626626')
# 将窗口调整最大
driver.maximize_window()
# 休息5s
time.sleep(1)
current_window_1 = driver.current_window_handle
print(current_window_1)

CDwindow-DF03E26EA9CBFF9D971A14943C556A1B


In [4]:
#找到页面上方登陆按钮并点击_OK
login = driver.find_element_by_xpath("//*[@id=\"headerwarp\"]/div[2]/div[1]/div/a[1]")
login.click()
time.sleep(2)
#选择返回电脑登陆，用户名输入法模式
userpw = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[1]/a[2]')
userpw.click()
time.sleep(2)
#输入用户名密码并提交
username = driver.find_element_by_xpath('//*[@id="username"]')
username.send_keys(name)
password = driver.find_element_by_xpath('//*[@id="user"]/div[1]/div[1]/div[1]/div[2]/input')
password.send_keys(passwd)
submit = driver.find_element_by_xpath('//*[@id="user"]/div[1]/div[3]/button')
submit.click()
#弹出的验证码还需要人工在浏览器中点击确认
#
#
#
#
#
#
#
#

In [6]:
tree = etree.HTML(driver.page_source) 
user = tree.xpath('//div[@class="auth"]/a/text()')#用户名
#print(user)
infc = tree.xpath('//div[@class="info clearfix"]')#
#for i in range(0, len(user)):
#    print(infc[i].xpath('string(.)').strip())
jifen    = tree.xpath('//div[@class="user_atten"]/ul/li[1]/div/a/text()')
depiao   = tree.xpath('//div[@class="user_atten"]/ul/li[2]/div/a/text()')
dingdang = tree.xpath('//div[@class="user_atten"]/ul/li[3]/div/a/text()')
    
content = tree.xpath('//td[@class="postbody"]')#string(.)看作路径，需要.xpath二次“查询”


#### 4. 保存爬取的内容

In [17]:
results = []
for i in range(0, len(user)-1):
    detail = "(【"+infc[i].xpath('string(.)').strip()+"】积分-"+jifen[i].strip()+"、得票-"+depiao[i].strip()+"、丁当-"+dingdang[i].strip()+ "): \n" 
    results.append(user[i].strip() + detail+ content[i].xpath('string(.)').strip())

In [21]:
# 打印爬取的结果
for i,result in zip(range(0, len(user)),results):
    print("user"+ str(i+1) + "  " + result)
    print("*"*50)

user1  楼医生(【常驻站友】积分-3、得票-2、丁当-182): 
我遇到一个“怪”病人，向大家请教。她，42岁。反复惊吓后晕厥30余年。每次受响声惊吓后发生跌倒，短暂意识丧失。无逆行性遗忘，无抽搐，无口吐白沫，无大小便失禁。多次跌倒致外伤。婴儿时有惊厥史。入院查体无殊。ECG、24小时动态心电图无殊；头颅MRI示小软化灶；脑电图无殊。入院后有数次类似发作。请问该患者该做何诊断，还需做什么检查，治疗方案怎样？
**************************************************
user2  lion000(【丁香园准中级站友】积分-43、得票-9、丁当-82): 
从发作的症状上比较符合血管迷走神经性晕厥，直立倾斜试验能协助诊断。在行直立倾斜实验前应该做常规的体格检查、ECG、UCG、holter和X-ray胸片除外器质性心脏病。贴一篇“口服氨酰心安和依那普利治疗血管迷走性晕厥的疗效观察”作者：林文华 任自文 丁燕生http://www.ccheart.com.cn/ccheart_site/Templates/jieru/200011/1-1.htm
**************************************************
user3  xghrh(【丁香园荣誉版主】积分-68、得票-19、丁当-412): 
同意lion000版主的观点：如果此患者随着年龄的增长，其发作频率逐渐减少且更加支持，不知此患者有无这一特点。入院后的HOLTER及血压监测对此患者只能是一种安慰性的检查，因在这些检查过程中患者发病的机会不是太大，当然不排除正好发作的情况。对此患者应常规作直立倾斜试验，如果没有诱发出，再考虑有无可能是其他原因所致的意识障碍，如室性心动过速等，但这需要电生理尤其是心腔内电生理的检查，毕竟是有一种创伤性方法。因在外地，下面一篇文章可能对您有助，请您自己查找一下。心理应激事件诱发血管迷走性晕厥1例 ，杨峻青、吴沃栋、张瑞云，中国神经精神疾病杂志， 2002 Vol.28 No.2
**************************************************
user4  keys(【常驻站友】积分-8、得票-3、丁当-141): 
该例不排除精神因素导致的，因为每次均在受惊

In [22]:
driver.quit()